## Import Dependencies

In [2]:
import csv
from pathlib import Path
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
# from sklearn import preprocessing
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression, RidgeCV
# from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas_profiling as pp
from settings import *
# from settings import DATA_DIR

In [3]:
download_dir = 'D:\Performance_All'
download_dir

'D:\\Performance_All'

In [4]:
DATA_DIR

'data'

In [5]:
outputFileName = 'FMPerfProcessedALL.csv'
outpath = "/".join([DATA_DIR,outputFileName])
outpath

'data/FMPerfProcessedALL.csv'

# Get the Single-Family Loan Performance Data Files from Fannie Mae

From the Fannie Mae [website:](https://loanperformancedata.fanniemae.com/lppub/index.html#Single-Family_Loan_Performance_Data_Files) we downloaded the Performance dataset (26GB zip file)

The [Performance file layout](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf) was as follows:
![Performance File Layout](images/PerformanceFileLayout.jpg)

In [6]:
# Create mapping for the header of the files based on above file format
allheaderline = ['id','rptPeriod','sellerName','currIntRate','currUPB','loanAge','monMatur','adjMonMatur',\
              'maturDate','mSA','deliqStatus','modFlag','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']
colchoices = [0,1,3,4,5,6,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
headerline = ['id','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
              'mSA','deliqStatus','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']

In [ ]:
# Loop through each file from the unzipped download file - which gave a set of 76 individual files

#Create an empty array to hold dataframes to later concatonate
li = []

#Loop through every file in the directory and append into array
for in_path in Path(download_dir).glob('Performance*.txt'):
    df = pd.read_csv(in_path, sep="|", index_col=None, header=None, usecols=colchoices)
    df.columns=headerline
    df = df.set_index('id')
    # Isolate the deliqStatus column to speed up aggregation calculations
    deliq = df[['deliqStatus']].copy()
    # deliqStatus has one value of 'X' amongst integer values and NaN values
    # First replace 'X' values with NaN
    deliq['deliqStatus'] = deliq['deliqStatus'].replace('X',np.nan).astype(float)
    # Determine the number of zero (0) values representing successful payments for a loan
    deliqGood = deliq[deliq['deliqStatus'] == 0].groupby('id').count()
    # Determine the number of values > 0 representing delinquent payments for a loan
    deliqBad = deliq[deliq['deliqStatus'] > 0].groupby('id').count()
    # Determine the max() of deliqStatus representing highest numberdelinquent payments in a row for a loan
    deliqMax = deliq[deliq['deliqStatus'] > 0].groupby('id').max()
#     # Select only rows that have a zeroBalCode with a value ## Commented out to include ALL records
#     zbc_df = df[df['zeroBalCode'].notnull()]
    df = df.drop(columns=['deliqStatus'])
    #Merge the zbc_df with the deliqGood
    zbc_merge = pd.merge(df, deliqGood, on=['id'])
    #Merge the zbc_df with the deliqBad
    zbc_merge = pd.merge(zbc_merge, deliqBad, on=['id'])
    #Merge the zbc_df with the deliqMax
    zbc_merge = pd.merge(zbc_merge, deliqMax, on=['id'])
    # Rename all the deqliq* columns
    zbc_merge = zbc_merge.rename(columns={'deliqStatus_x':'deliqGood','deliqStatus_y':'deliqBad','deliqStatus':'deliqMax'})
    # Add the zbc_merge df to the li list to be able to concat them later
    li.append(zbc_merge)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
len(li)

78

In [8]:
#Concatonate all the array elements into one giant dataframe
processed_df = pd.concat(li, axis=0)
#df.columns = allheaderline

In [9]:
processed_df.shape

(4054042, 28)

In [10]:
processed_df.head()

,rptPeriod,currIntRate,currUPB,loanAge,monMatur,mSA,zeroBalCode,zeroBalDate,lastPdInstDate,forecloDate,...,repurchProceed,otherForecloProceed,nonIntUPB,prinForgivBal,repurchMakeWholeProceedFlg,forecloPrinWriteOffAmnt,servActivIndicator,deliqGood,deliqBad,deliqMax
id,,,,,,,,,,,,,,,,,,,,,
100021703104,01/01/2003,8.000,109698.66,34,326.0,35980,1.0,01/2003,NaN,NaN,...,NaN,NaN,NaN,NaN,N,NaN,NaN,33,1,1.0
100023274028,09/01/2002,8.000,127430.15,31,329.0,12060,1.0,09/2002,NaN,NaN,...,NaN,NaN,NaN,NaN,N,NaN,NaN,20,11,4.0
100033892603,08/01/2003,8.125,102258.47,42,318.0,33100,1.0,08/2003,NaN,NaN,...,NaN,NaN,NaN,NaN,N,NaN,NaN,40,1,1.0
100048724218,12/01/2001,7.750,148631.79,23,337.0,13460,1.0,12/2001,NaN,NaN,...,NaN,NaN,NaN,NaN,N,NaN,NaN,22,1,1.0
100057706656,03/01/2001,8.250,135031.31,14,346.0,17140,6.0,03/2001,NaN,NaN,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,13,1,1.0


In [11]:
#Review the columns for datatypes and how many non-null values 
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4054042 entries, 100021703104 to 998487923072
Data columns (total 28 columns):
rptPeriod                     object
currIntRate                   float64
currUPB                       float64
loanAge                       int64
monMatur                      float64
mSA                           int64
zeroBalCode                   float64
zeroBalDate                   object
lastPdInstDate                object
forecloDate                   object
dispDate                      object
forecloCost                   float64
propRepCost                   float64
recovCosts                    float64
miscCost                      float64
holdTaxCost                   float64
saleProceed                   float64
credEnhProceed                float64
repurchProceed                float64
otherForecloProceed           float64
nonIntUPB                     float64
prinForgivBal                 float64
repurchMakeWholeProceedFlg    object
foreclo

### Convert the origDate,firstPmtDate columns from strings to datetime datatypes
Since this will be such a large dataset I tested a number of methods to change datatypes

In [12]:
processed_df.shape

(4054042, 28)

In [13]:
processed_df.dispDate.head(30)

id
100021703104           NaN
100023274028           NaN
100033892603           NaN
100048724218           NaN
100057706656           NaN
100094030068           NaN
100100228738           NaN
100118647578           NaN
100119700071           NaN
100135299882           NaN
100166340980           NaN
100181385102           NaN
100187515017           NaN
100211950135           NaN
100260403844           NaN
100289940808           NaN
100297620817           NaN
100339426081           NaN
100366510395           NaN
100379332657           NaN
100392250518           NaN
100416816452           NaN
100442840341           NaN
100444558979           NaN
100507009460           NaN
100530507310           NaN
100536680291           NaN
100557946796    05/01/2004
100563946369           NaN
100579898395    05/01/2004
Name: dispDate, dtype: object

In [14]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4054042 entries, 100021703104 to 998487923072
Data columns (total 28 columns):
rptPeriod                     object
currIntRate                   float64
currUPB                       float64
loanAge                       int64
monMatur                      float64
mSA                           int64
zeroBalCode                   float64
zeroBalDate                   object
lastPdInstDate                object
forecloDate                   object
dispDate                      object
forecloCost                   float64
propRepCost                   float64
recovCosts                    float64
miscCost                      float64
holdTaxCost                   float64
saleProceed                   float64
credEnhProceed                float64
repurchProceed                float64
otherForecloProceed           float64
nonIntUPB                     float64
prinForgivBal                 float64
repurchMakeWholeProceedFlg    object
foreclo

In [15]:
#processed_df['rptPeriod'] = pd.to_datetime(processed_df['rptPeriod'], format='%Y/%m/%d')
#processed_df['lastPdInstDate'] = pd.to_datetime(processed_df['lastPdInstDate'], format='%Y/%m/%d')
#processed_df['zeroBalDate'] = pd.to_datetime(processed_df['zeroBalDate'], format='%m/%Y')
#processed_df['forecloDate'] = pd.to_datetime(processed_df['forecloDate'], format='%m/%d/%Y')
processed_df['dispDate'] = pd.to_datetime(processed_df['dispDate'], format='%m/%d/%Y')

In [16]:
# Change all occurences of columns with 'Y' and 'N' to 0 and 1
processed_df.replace('Y',1, inplace=True)
processed_df.replace('N',0, inplace=True)

In [17]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4054042 entries, 100021703104 to 998487923072
Data columns (total 28 columns):
rptPeriod                     object
currIntRate                   float64
currUPB                       float64
loanAge                       int64
monMatur                      float64
mSA                           int64
zeroBalCode                   float64
zeroBalDate                   object
lastPdInstDate                object
forecloDate                   object
dispDate                      datetime64[ns]
forecloCost                   float64
propRepCost                   float64
recovCosts                    float64
miscCost                      float64
holdTaxCost                   float64
saleProceed                   float64
credEnhProceed                float64
repurchProceed                float64
otherForecloProceed           float64
nonIntUPB                     float64
prinForgivBal                 float64
repurchMakeWholeProceedFlg    int64


# Write the result df to csv

In [18]:
processed_df.to_csv(outpath)